In [1]:
import torch
from torch.utils.data import Dataset , DataLoader
import cv2
import PIL
import os
from torchvision.transforms import Compose, ToTensor, Normalize, Resize
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard.writer import SummaryWriter
import shutil
from sklearn.metrics import accuracy_score , confusion_matrix
import numpy as np

In [2]:
if torch.cuda.is_available(): 
 dev = "cuda:0" 
else: 
 dev = "cpu" 
device = torch.device(dev) 
device

device(type='cuda', index=0)

In [3]:
class AnimalDataset(Dataset):
    def __init__(self, root = "animals_v2/animals" , train = True , transform = None):
        self.categories = ["butterfly", "cat", "chicken", "cow", "dog", "elephant", "horse", "sheep", "spider", "squirrel"]
        if train : 
            data_path = os.path.join(root , "train")
        else:
            data_path = os.path.join(root , "test")

        self.image_paths = []
        self.labels = []

        for category in self.categories:
            category_path = os.path.join(data_path , category)
            for image_name in os.listdir(category_path):
                image_path = os.path.join(category_path, image_name)
                self.image_paths.append(image_path)
                self.labels.append(self.categories.index(category))
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, item):
        image = cv2.imread(self.image_paths[item])
        if self.transform:
            image = self.transform(image)
        label = self.labels[item]
        return image , label

In [4]:
data_transform = Compose([
    ToTensor(),
    Resize((256,256)),
  ])

In [5]:
train_set = AnimalDataset(train = True , transform = data_transform)
test_set = AnimalDataset(train = False , transform = data_transform)

In [6]:
len(train_set)

23699

In [7]:
train_loader = DataLoader(train_set , batch_size = 16 , shuffle = True , drop_last = True)
test_loader = DataLoader(test_set , batch_size = 16 , shuffle = False )

In [8]:
num_classes = len(train_set.categories)
num_classes

10

In [30]:
class My_Model(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=0),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=0),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride = 2)
        )#126
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=0),
            nn.BatchNorm2d(num_features=32),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=0),
            nn.BatchNorm2d(num_features=32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride = 2)
        )#61
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=0),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=0),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride = 2)
        )#28
        self.layer4 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=0),
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=0),
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride = 2)
        )#
        self.layer5 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=0),
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=0),
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride = 2)
        )
        
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4*4*256, 2048),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(2048, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU()
        )
        self.fc3= nn.Sequential(
            nn.Linear(4096, num_classes))
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        out = F.softmax(out, dim=1)
        return out

In [31]:
device

device(type='cuda', index=0)

In [32]:
model = My_Model(num_classes).to(device)

In [33]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.7)
num_epochs = 80
num_iter = len(train_loader)

In [1]:
log_path = "tensorboard"
if os.path.isdir(log_path):
    shutil.rmtree(log_path)
os.mkdir(log_path)
writer = SummaryWriter(log_path)

if not os.path.isdir("trained_models_1"):
    os.mkdir("trained_models_1")
writer = SummaryWriter(log_path)

best_acc = 0

for epoch in range(num_epochs):
    model.train()
    losses = []
    for i, (images , labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        y_pred = model(images)
        loss = criterion(y_pred , labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_value = loss.item()
        print("Epoch {}/{} | Iteration {}/{} | Loss value : {}".format(epoch + 1 , num_epochs , i , num_iter, loss_value))
        losses.append(loss_value)
        writer.add_scalar("Train/Loss" , np.mean(losses), epoch*num_iter + i)

    model.eval()
    losses = []
    all_predictions = []
    all_gts = []
    with torch.inference_mode():
        for i, (images , labels) in enumerate(test_loader):
            images = images.to(device)
            labels = labels.to(device)

            pred = model(images)
            max_idx = torch.argmax(pred , 1)
            loss = criterion(pred , labels)
            losses.append(loss.item())
            all_gts.extend(labels.tolist())
            all_predictions.extend(max_idx.tolist())
    writer.add_scalar("Val/Loss" , np.mean(losses), epoch)
    acc = accuracy_score(all_gts , all_predictions)
    writer.add_scalar("Val/Accuracy", acc , epoch)
    conf_matrix = confusion_matrix(all_gts , all_predictions)
    checkpoint = {
                "model": model.state_dict(),
                "optimizer": optimizer.state_dict(),
                "epoch": epoch,
                "best_acc": best_acc,
            }
    torch.save(checkpoint, os.path.join("trained_models_1", "last.pt"))
    if acc > best_acc:
        torch.save(checkpoint, os.path.join("trained_models_1", "best.pt"))
        best_acc = acc
        

NameError: name 'os' is not defined

In [ ]:
image = cv2.imread("sheep.jpg")
image = cv2.resize(image, (256, 256))
image = np.transpose(image, (2, 0, 1))
image = image / 255
image = np.expand_dims(image, 0)
image = torch.from_numpy(image).to(device).float()


In [69]:
image.shape

torch.Size([1, 3, 256, 256])

In [70]:
model.eval()
with torch.inference_mode():
    pred = model(image).to(device)


In [71]:
pred

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 1.6746e-24, 0.0000e+00, 1.4013e-45,
         0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00]], device='cuda:0')

In [72]:
x = pred.argmax()

In [73]:
x

tensor(7, device='cuda:0')

In [74]:
train_set.categories[x]

'sheep'

In [75]:
train_set.categories

['butterfly',
 'cat',
 'chicken',
 'cow',
 'dog',
 'elephant',
 'horse',
 'sheep',
 'spider',
 'squirrel']